In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
path2csv = './csv/'
params = ['threshold-val', 'percent-grammar-1']

In [3]:
def csv_to_df(path2csv):
    dff_all_from_csv = []
    print(path2csv)
    
    for root, dirs, files in os.walk(path2csv):
        print(root, files)
        for file in files:
            if not file.endswith('.csv'):
                continue
            df = pd.read_csv(os.path.join(root, file), skiprows=range(0,6))
            df.set_index(['[run number]', '[step]'], inplace=True)
            dff_all_from_csv.append(df)
    return dff_all_from_csv

In [4]:
df_all_from_csv = csv_to_df(path2csv)

./csv/
./csv/ ['Language Change_converge_stop experiment8-reward-alpha-table.csv', 'Language Change_converge_stop experiment1-individual-perc-table.csv', 'Language Change_converge_stop experiment3-threshold-perc-table.csv', 'Language Change_converge_stop experiment2-threshold-perc-thres-table.csv', 'Language Change_converge_stop experiment4-reward-perc-table.csv', 'Language Change_converge_stop experiment11-individual-population-table.csv', 'Language Change_converge_stop experiment9-reward-population-table.csv', 'Language Change_converge_stop experiment6-threshold-sink-state-table.csv', 'Language Change_converge_stop experiment5-threshold-thres-table.csv', 'Language Change_converge_stop experiment7-reward-logistic-table.csv', 'Language Change_converge_stop experiment10-threshold-population-table.csv']


In [5]:
df_all_from_csv[0]

logistic?  alpha update-algorithm  num-nodes  \
[run number] [step]                                                 
6            0            True   0.00           reward        100   
7            0            True   0.00           reward        100   
6            1            True   0.00           reward        100   
7            1            True   0.00           reward        100   
             2            True   0.00           reward        100   
...                        ...    ...              ...        ...   
543          977          True   0.05           reward        100   
             978          True   0.05           reward        100   
             979          True   0.05           reward        100   
             980          True   0.05           reward        100   
             981          True   0.05           reward        100   

                     percent-grammar-1  sink-state-1?  threshold-val  \
[run number] [step]                                                    
6            0                      60           True            0.3   
7            0                      60           True            0.3   
6            1                      60           True            0.3   
7            1                      60           True            0.3   
             2                      60           True            0.3   
...                                ...            ...            ...   
543          977                    60           True            0.3   
             978                    60           True            0.3   
             979                    60           True            0.3   
             980                    60           True            0.3   
             981                    60           True            0.3   

                     final-grammar1-distribution  
[run number] [step]                               
6            0                          0.600000  
7            0                          0.600000  
6            1                          0.600147  
7            1                          0.600074  
             2                          0.599624  
...                                          ...  
543          977                        0.989519  
             978                        0.989631  
             979                        0.989742  
             980                        0.989851  
             981                        0.989959  

[2454790 rows x 8 columns]

In [6]:
def t_test(vals1, vals2):
    _, p_val = stats.ttest_ind(vals1, vals2)
    if p_val < 0.05:
        print("p-value is significant: ", p_val)
        return p_val, True
    else:
        #return
        print("p-value is NOT significant: ", p_val)
        return p_val, False

In [7]:
# statistical relevance: t-test
from scipy import stats

val2testdf = {'threshold-val':[0.1, 0.9], 'percent-grammar-1':[10,30,60],'alpha':[0.0,0.05],'logistic?':[0, 1], 'num-nodes':[10, 50, 100]}
save_df = pd.DataFrame(columns=['infection_type', 'parameter', 'distr1_ID', 'distr2_ID', 'p-value', 'stat_relevant?'])

for experiment_df in df_all_from_csv:
    # determine type of grammar spread
    infection_type = experiment_df['update-algorithm'].iloc[0]

    # set parameters tested in experiment
    if infection_type == 'individual' or infection_type == 'threshold':
        params = ['threshold-val', 'percent-grammar-1', 'num-nodes']
    else:
        params = ['alpha', 'logistic?', 'num-nodes']
    
    for p in params:
        vals, counts = np.unique(experiment_df[p], return_counts=True)
        if len(vals) == 1:
            # if only one value for parameter, no need to plot
            continue

        vals2test = val2testdf[p]
        steps_p = {}
        for val in vals2test:
            temp_df = experiment_df.reset_index(inplace=False).groupby([p, "[run number]"]).max()["[step]"].reset_index(inplace=False)
            steps_p[val] = temp_df[temp_df[p] == val]['[step]'].values
        #print(vals2test, steps_p)
        min =  np.min(vals2test)
        max = np.max(vals2test)
        # thresholds p 0.1 vs 0.9
        print(f"\n{p}-Experiment: {min} vs {max} ")#, experiment_df.index[0][0])
        p_val, stat_sig = t_test(steps_p[min], steps_p[max])
        save_df = pd.concat([save_df, pd.DataFrame({'infection_type':infection_type, 'parameter':p, 'distr1_ID':min, 'distr2_ID':max, 'p-value':p_val, 'stat_relevant?':stat_sig}, index=[0])], ignore_index=True)

        if len(vals2test) == 2:
            continue
        # thresholds p 0.1 vs 0.6
        print(f"\n{p}-Experiment: {min} vs {vals2test[1]} ")#, experiment_df.index[0][0])
        p_val, stat_sig = t_test(steps_p[min], steps_p[vals2test[1]])
        save_df = pd.concat([save_df, pd.DataFrame({'infection_type':infection_type, 'parameter':p, 'distr1_ID':min, 'distr2_ID':vals2test[1], 'p-value':p_val, 'stat_relevant?':stat_sig}, index=[0])], ignore_index=True)

        # thresholds p 0.6 vs 0.9
        print(f"\n{p}-Experiment: {vals2test[1]} vs {max} ")#, experiment_df.index[0][0])
        p_val, stat_sig = t_test(steps_p[vals2test[1]], steps_p[max])
        save_df = pd.concat([save_df, pd.DataFrame({'infection_type':infection_type, 'parameter':p, 'distr1_ID':vals2test[1], 'distr2_ID':max, 'p-value':p_val, 'stat_relevant?':stat_sig}, index=[0])], ignore_index=True)


alpha-Experiment: 0.0 vs 0.05 
p-value is significant:  1.9585727824909736e-69

percent-grammar-1-Experiment: 10 vs 60 
p-value is significant:  0.00011725853915384903

percent-grammar-1-Experiment: 10 vs 30 
p-value is significant:  0.0014791401167298733

percent-grammar-1-Experiment: 30 vs 60 
p-value is NOT significant:  0.2964430652325921

percent-grammar-1-Experiment: 10 vs 60 
p-value is significant:  1.0032258535350447e-70

percent-grammar-1-Experiment: 10 vs 30 
p-value is significant:  1.354758849304321e-05

percent-grammar-1-Experiment: 30 vs 60 
p-value is significant:  1.1076048643446832e-16


/var/folders/hk/dwphr0p14vn00wrnlw8bxjpw0000gn/T/ipykernel_7750/341101331.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  save_df = pd.concat([save_df, pd.DataFrame({'infection_type':infection_type, 'parameter':p, 'distr1_ID':min, 'distr2_ID':max, 'p-value':p_val, 'stat_relevant?':stat_sig}, index=[0])], ignore_index=True)
/Users/klara/Developer/Uni/SoSe24/I2ABM_SoSe24/I2ABM_venv/lib/python3.11/site-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)



threshold-val-Experiment: 0.1 vs 0.9 
p-value is significant:  6.709906385872263e-270

percent-grammar-1-Experiment: 10 vs 60 
p-value is significant:  3.2234769561784984e-26

percent-grammar-1-Experiment: 10 vs 30 
p-value is significant:  1.772178419203113e-05

percent-grammar-1-Experiment: 30 vs 60 
p-value is significant:  2.165981639392901e-10

num-nodes-Experiment: 10 vs 100 
p-value is significant:  9.331710779776049e-10

num-nodes-Experiment: 10 vs 50 
p-value is significant:  1.4387758783072337e-11

num-nodes-Experiment: 50 vs 100 
p-value is significant:  2.6665261509910782e-05

num-nodes-Experiment: 10 vs 100 
p-value is NOT significant:  0.30419513585396796

num-nodes-Experiment: 10 vs 50 
p-value is NOT significant:  0.8659807104875764

num-nodes-Experiment: 50 vs 100 
p-value is NOT significant:  0.16664452392251142

threshold-val-Experiment: 0.1 vs 0.9 
p-value is significant:  2.8371560761165118e-251

logistic?-Experiment: 0 vs 1 
p-value is NOT significant:  0.1066524

/Users/klara/Developer/Uni/SoSe24/I2ABM_SoSe24/I2ABM_venv/lib/python3.11/site-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


Language Change_converge_stop experiment1-individual-perc-table.csv: percent-grammar-1 is significant for the time it takes until convergence

In [8]:
save_df

,infection_type,parameter,distr1_ID,distr2_ID,p-value,stat_relevant?
0,reward,alpha,0.0,0.05,1.958573e-69,True
1,individual,percent-grammar-1,10.0,60.00,1.172585e-04,True
2,individual,percent-grammar-1,10.0,30.00,1.479140e-03,True
3,individual,percent-grammar-1,30.0,60.00,2.964431e-01,False
4,threshold,percent-grammar-1,10.0,60.00,1.003226e-70,True
5,threshold,percent-grammar-1,10.0,30.00,1.354759e-05,True
6,threshold,percent-grammar-1,30.0,60.00,1.107605e-16,True
7,threshold,threshold-val,0.1,0.90,6.709906e-270,True
8,threshold,percent-grammar-1,10.0,60.00,3.223477e-26,True
9,threshold,percent-grammar-1,10.0,30.00,1.772178e-05,True


In [9]:
save_df.to_csv('./plots/statistical_relevance/t_test_results.csv')